<h3><center>Black Scholes Formula<br>
CUNY SPS MSDS<br>
Professor Samuel Gralnick<br>
Team Sugar Cane: Euclid Zhang, Jie Zou, Zhenni Xie<br>
Fall 2022</center></h3>

In this analysis, we calculate the prices of the european call options and european put options of the equities in our investment portfolio.

In [23]:
#Loading packages

import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import pandas_datareader.data as pdr
import pandas as pd
import scipy.stats as stats
import math

In the calculations, we calculate the returns and volatilities using the assets' prices for 1 calendar year. The risk-free rate is from https://www.marketwatch.com/investing/bond/tmubmusd01y?countrycode=bx. We assume that there are 251 trade days in a calendar year, this is used to annualize the returns and standard deviations of the assets. We will set the strike price equal to 80% of the current price for the option calculations. We will calculate the prices of option with expiration period 1 calendar year.

In [24]:
#The start date and end date of the assets' prices used for the calculations

start = datetime.strptime('2021-10-22', '%Y-%m-%d')
end = datetime.strptime('2022-10-22', '%Y-%m-%d')

#Seting up parameters for the option calculations. 

#1-year risk free rate from https://www.marketwatch.com/investing/bond/tmubmusd01y?countrycode=bx
risk_free_rate = 0.04597
#251 trade days in a year, used to annualized the return and standard deviation
n_trade_days = 251
#percentage of the current price, used as the strike price of the options
strike_price_pct = 0.8
#Expiration period of the option, 1 year
time_period = 1

Our investment portfolio includes the following assets, with purchasing prices (close prices) from 9/16/2022. In this analysis, **we will not include the U.S. 30 Year Treasury bonds in our calculation**

In [25]:
#Load the infromation of the assets in our portfolio

assets = pd.read_csv("portfolio assets.csv")
assets

,Ticker,Type,Sector,Price,Units,Amount
0,US30Y,U.S. 30 Year Treasury,NaN,90.44,467317.00,42262981.19
1,HSON,Common Stock,COMMERCIAL SERVICES,34.95,36138.24,1263031.49
2,GOGO,Common Stock,COMMUNICATIONS,13.32,94822.19,1263031.57
3,JAKK,Common Stock,CONSUMER DURABLES,23.37,54044.99,1263031.42
4,CALM,Common Stock,CONSUMER NON-DURABLES,56.85,22216.91,1263031.33
5,HRB,Common Stock,CONSUMER SERVICES,44.93,28111.09,1263031.27
6,HDSN,Common Stock,DISTRIBUTION SERVICES,7.94,159071.98,1263031.52
7,BELFA,Common Stock,ELECTRONIC TECHNOLOGY,28.52,44285.81,1263031.30
8,ARLP,Common Stock,ENERGY MINERALS,23.88,52890.76,1263031.35
9,CI,Common Stock,HEALTH SERVICES,290.35,4350.03,1263031.21


We then retrieve the prices and volumes of the common stocks and ETFs in our portfolio, using the yfinance package

In [26]:
#retrieve the prices and volumes of the common stocks and ETFs in our portfolio, using the yfinance package

yf.pdr_override()

tickers = list(assets['Ticker'])[1:]

historical_data = pdr.DataReader(tickers, start, end , data_source='yahoo')

historical_data = historical_data.reset_index()
historical_data.Date = historical_data.Date.apply(datetime.date)
historical_data.set_index('Date', inplace = True)

close_prices = historical_data['Adj Close']
close_prices

[*********************100%***********************]  29 of 29 completed


,ARLP,ASC,AZPN,BELFA,BSM,CALM,CF,CI,CSL,DIG,...,JAKK,LNG,MUSA,SRTS,TBT,TMV,TTT,TYO,UUP,YCS
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-22,10.832505,4.06,157.000000,13.568584,11.017209,35.357189,58.687115,212.543518,225.766342,20.777027,...,11.660000,104.169579,175.914780,4.03,18.260000,61.860001,34.900002,8.87,25.120001,40.430000
2021-10-25,11.363419,4.07,154.869995,13.796044,11.107738,35.426037,58.156017,211.777435,229.244537,21.391468,...,11.570000,106.871048,175.387756,4.01,18.320000,62.130001,35.060001,8.84,25.160000,40.619999
2021-10-26,11.708047,3.95,153.889999,13.796044,11.008157,35.435871,57.575733,211.826553,227.936508,21.631792,...,11.670000,106.059616,167.601822,4.02,18.020000,60.639999,34.230000,8.82,25.209999,40.900002
2021-10-27,11.428618,4.02,150.149994,13.103771,10.962893,35.544056,55.805382,209.597000,221.029648,20.469807,...,12.020000,104.753410,164.747955,3.93,17.370001,57.330002,32.290001,8.66,25.200001,40.680000
2021-10-28,11.177134,4.03,154.910004,13.153220,11.315950,36.084984,56.621712,211.905121,222.852966,20.752251,...,10.970000,103.476898,162.073090,3.96,17.490000,57.930000,32.590000,8.73,25.049999,40.520000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-17,22.600000,11.04,252.330002,29.010000,17.000000,60.639999,98.879997,289.920013,291.579987,38.169998,...,18.719999,171.600006,284.010010,12.23,34.610001,151.490005,85.989998,14.18,30.150000,69.139999
2022-10-18,23.020000,11.61,253.000000,29.180000,17.200001,59.610001,100.239998,293.739990,293.450012,38.630001,...,18.719999,167.970001,282.940002,12.12,34.400002,150.539993,85.500000,14.13,30.129999,69.250000
2022-10-19,22.639999,11.89,248.889999,29.490000,17.549999,60.680000,100.660004,293.079987,285.959991,40.820000,...,18.350000,170.750000,282.179993,11.93,35.669998,158.619995,90.220001,14.53,30.350000,69.930000


In the option price calculations, we use the log returns

In [27]:
#Calculate the daily log returns

log_return = np.log(close_prices / close_prices.shift(1)).dropna()

In [28]:
#Define a function to calculate the call option

def european_call(stock_price, strike_price, risk_free_rate, time_period, volatility):
  d_1 = (math.log(stock_price/strike_price) + (risk_free_rate + 0.5*(volatility**2))*time_period)/(volatility*math.sqrt(time_period))
  d_2 = d_1 - volatility*math.sqrt(time_period)
  option_price = stock_price * stats.norm.cdf(d_1) - strike_price * math.exp(-risk_free_rate*time_period) * stats.norm.cdf(d_2)
  return option_price

In [29]:
#Define a function to calculate the put option

def european_put(stock_price, strike_price, risk_free_rate, time_period, volatility):
  d_1 = (math.log(stock_price/strike_price) + (risk_free_rate + 0.5*(volatility**2))*time_period)/(volatility*math.sqrt(time_period))
  d_2 = d_1 - volatility*math.sqrt(time_period)
  option_price =  strike_price * math.exp(-risk_free_rate*time_period) * stats.norm.cdf(-d_2) - stock_price * stats.norm.cdf(-d_1)
  return option_price

In [30]:
#Gather the information for the option price calculation for each asset

asset_info = pd.DataFrame({"stock_price" : close_prices.iloc[len(close_prices)-1],
              "strike_price" : close_prices.iloc[len(close_prices)-1] * strike_price_pct,
              "risk_free_rate" : math.log(1+risk_free_rate),
              "time_period" : time_period,
              "volatility" : math.sqrt(n_trade_days) * log_return.std()})

In [31]:
#Calculate the prices of the call and put options

asset_info['call_price'] = asset_info.apply(lambda x: european_call(x['stock_price'], 
                                         x['strike_price'],
                                         x['risk_free_rate'],
                                         x['time_period'],
                                         x['volatility']), axis=1)
asset_info['put_price'] = asset_info.apply(lambda x: european_put(x['stock_price'], 
                                         x['strike_price'],
                                         x['risk_free_rate'],
                                         x['time_period'],
                                         x['volatility']), axis=1)

The following is the result of our calculations.

In [32]:
#Result of the calculations

asset_info

,stock_price,strike_price,risk_free_rate,time_period,volatility,call_price,put_price
ARLP,22.760000,18.208000,0.044945,1,0.479219,7.044185,1.691950
ASC,11.690000,9.352000,0.044945,1,0.591874,4.008553,1.259536
AZPN,253.639999,202.912000,0.044945,1,0.363252,70.333709,10.687801
BELFA,30.209999,24.167999,0.044945,1,0.482695,9.380423,2.276248
BSM,17.629999,14.103999,0.044945,1,0.343947,4.801065,0.655200
CALM,58.689999,46.951999,0.044945,1,0.321967,15.661778,1.860256
CF,107.370003,85.896002,0.044945,1,0.494922,33.721635,8.472537
CI,301.339996,241.071997,0.044945,1,0.267241,76.662443,5.799417
CSL,273.880005,219.104004,0.044945,1,0.305211,71.989296,7.583754
DIG,42.939999,34.351999,0.044945,1,0.704232,16.189409,6.091652


We observe that for a call option with strike price 20% lower than the current asset price, the option price is more than 
20% of the current asset price. This is reasonable since the call option is in the money and the option price must be higher than the asset price - strike price as of now.

The price of the put options are lower than the call options since the call options are out of the money.

We then calculate the ratio of option price to the current asset price, as showed below

In [33]:
#Calculate the ratio of option price to the current asset price

asset_info['strike_to_price'] = asset_info['strike_price'] / asset_info['stock_price']
asset_info['call_to_price'] = asset_info['call_price'] / asset_info['stock_price']
asset_info['put_to_price'] = asset_info['put_price'] / asset_info['stock_price']
asset_info

,stock_price,strike_price,risk_free_rate,time_period,volatility,call_price,put_price,strike_to_price,call_to_price,put_to_price
ARLP,22.760000,18.208000,0.044945,1,0.479219,7.044185,1.691950,0.8,0.309498,0.074339
ASC,11.690000,9.352000,0.044945,1,0.591874,4.008553,1.259536,0.8,0.342904,0.107745
AZPN,253.639999,202.912000,0.044945,1,0.363252,70.333709,10.687801,0.8,0.277297,0.042138
BELFA,30.209999,24.167999,0.044945,1,0.482695,9.380423,2.276248,0.8,0.310507,0.075348
BSM,17.629999,14.103999,0.044945,1,0.343947,4.801065,0.655200,0.8,0.272324,0.037164
CALM,58.689999,46.951999,0.044945,1,0.321967,15.661778,1.860256,0.8,0.266856,0.031696
CF,107.370003,85.896002,0.044945,1,0.494922,33.721635,8.472537,0.8,0.314069,0.078910
CI,301.339996,241.071997,0.044945,1,0.267241,76.662443,5.799417,0.8,0.254405,0.019245
CSL,273.880005,219.104004,0.044945,1,0.305211,71.989296,7.583754,0.8,0.262850,0.027690
DIG,42.939999,34.351999,0.044945,1,0.704232,16.189409,6.091652,0.8,0.377024,0.141864


The option price of a highly volatile asset is extremely expansive. For example, the price of the call option for HUDI is more than 50% of the current asset price and the price of the put option is more than 30% of the current asset price. This indicates the asset is a risky one. However, with great risk comes great reward. Since the asset only take a very small portion of our portfolio, we can afford the risk and aim for a return boost to our portfolio.